In [10]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from zebrafish_ms2_paper.trace_analysis import extract_traces, binarize_trace, get_on_and_off_times, get_burst_durations, get_burst_inactive_durations, enforce_1spot_per_nucleus
from zebrafish_ms2_paper.ap_analysis import (bin_aps, get_ap_for_somites, 
                        filter_spots_by_distance_from_somite, 
                        filter_spots_by_perp_distance_from_ap_axis, 
                        get_ap_for_spots,
                        get_microns_between_ap_bins,
                        get_ap_um,
                        bin_aps_um)
from zebrafish_ms2_paper.utils import pboc_rc, style_axes, colors, fontsize
import pandas as pd
import pickle
from matplotlib import rcParams

In [2]:
%matplotlib qt

In [30]:
rcParams.update(pboc_rc)

In [91]:
fontsize = 11

In [3]:
"""specify the path to the folder of Supplemental Data Files"""
# REPLACE THIS WITH YOUR PATH
datadir = r'/home/brandon/Downloads/SupplementalDataFiles'

# define paths to particular files
path_to_spots = datadir + '/Supplemental_Data_File_2-Spots_Curated.pkl'
path_to_non_blank_timepoints = datadir + '/Supplemental_Data_File_3-Non-Blank_Time_Points.pkl'
path_to_ap = datadir + '/Supplemental_Data_File_6-Anterior-Posterior_Axis.pkl'


In [13]:
"""load all of the various datasets you need"""
df = pd.read_pickle(path_to_spots)

# coordinates of the ap axis
with open(path_to_ap, 'rb') as f:
    ap = pickle.load(f)

# correct time stamps
with open(path_to_non_blank_timepoints,'rb') as fp:
    non_blank_timepoints = pickle.load(fp)

t_arr = np.arange(0, 160)

# assign ap positions for all spots
df = get_ap_for_spots(df, ap)

In [27]:
"""result of collective period slowdown from Fig. 4"""
tissue_peaks = np.array([23.64834728,  48.99299716,  83.17459412])
tissue_periods = np.array([25.1439267 , 33.77065793, 35.51921158])
tissue_period_stds = np.array([1.7938441 , 4.34319991, 3.28815847])

In [25]:
"""region of ap axis considering"""
lower = 240 / 5          # microns / 5 microns per bin
upper = 315 / 5

In [48]:
traces = extract_traces(df, method='gauss3d_dog')

In [148]:
plt.close('all')

In [149]:
fig, axs = plt.subplots(4, 1, figsize=(6.37333333, 6.19333333))

In [150]:
x_min = 0
x_max = 100

In [151]:
"""re-plot tissue scale period rise"""
# ms2 periods
ax = axs[0]
ax.clear()
ax.errorbar(tissue_peaks, tissue_periods, tissue_period_stds, marker='o',
             markersize=12, color=colors['green'], capsize=4, capthick=2, linewidth=2, markerfacecolor=(colors['green'],) + (0.75,))

#ax.set_xlabel('time (min)', fontsize=fontsize)
ax.set_ylabel('tissue oscillation\nperiod (min)', fontsize=fontsize)
ax.set_xlim([x_min, x_max])
ax.set_xticks([0, 30, 60, 90], labels=[])
ax.set_yticks([20, 40])
ax.set_ylim([20,45])
ax = style_axes(ax, fontsize=fontsize)

## burst separations

In [152]:
"""extract individual bursts and its corresponding separation, time, and ap location"""
aps = []
quiets = []
event_times = []
for trace in traces:
    t_arr, inten_arr, nucleus = trace
    sub_df = df[df.nucleus_id == nucleus]
    state = binarize_trace(inten_arr, t_arr, thresh=1.0, window_size=3)
    on_times, off_times = get_on_and_off_times(state, t_arr)
   
    # inactive duration
    these_burst_quiets = get_burst_inactive_durations(on_times, off_times)
    these_reference_times = off_times[:len(these_burst_quiets)]
    these_aps = np.zeros(len(these_reference_times))
    these_aps_relative_to_somites = np.zeros(len(these_reference_times))
    for i, t in enumerate(these_reference_times):
        these_aps[i] = sub_df[np.abs(sub_df.t - t) == np.min(np.abs(sub_df.t - t))].ap.iloc[0]
        
    aps.extend(these_aps)
    quiets.extend(these_burst_quiets)
    event_times.extend(these_reference_times)

In [153]:
"""plot separations over time at fixed ap bins"""
these_quiets = [x for i, x in enumerate(quiets) if aps[i] > lower and aps[i] <= upper]
these_times = [x for i, x in enumerate(event_times) if aps[i] > lower and aps[i] <= upper]

coarse_time_bins = tissue_peaks#np.linspace(16, 119, 4)#10)
coarse_times = np.zeros_like(these_times)
for i in range(len(these_times)):
    coarse_times[i] = coarse_time_bins[np.abs(coarse_time_bins - these_times[i]) == np.min(np.abs(coarse_time_bins - these_times[i]))]
    
mean_quiets = np.zeros_like(coarse_time_bins)
std_quiets = np.zeros_like(coarse_time_bins)
for i in range(len(coarse_time_bins)):
    tmp_quiets = np.array(these_quiets)[coarse_times == coarse_time_bins[i]]
    mean_quiets[i] = np.mean(tmp_quiets)
    std_quiets[i] = np.std(tmp_quiets) #/ np.sqrt(len(tmp_quiets))
    
ax = axs[1]
ax.clear()
ax.plot(these_times, these_quiets, 'o', markerfacecolor=(0, 0, 0, 0.5), markeredgecolor='none', linewidth=0)
ax.errorbar(coarse_time_bins, mean_quiets, std_quiets, marker='o',
             markersize=12, color=colors['green'], capsize=4, capthick=2, linewidth=2, markerfacecolor=(colors['green'],) + (0.75,))
#ax.set_xlabel('time (min)', fontsize=fontsize)
ax.set_ylabel('burst separation\n(min)', fontsize=fontsize)
ax.set_xlim([x_min, x_max])
ax.set_xticks([0, 30, 60, 90], labels=[])
ax = style_axes(ax, fontsize=fontsize)


/tmp/ipykernel_2357556/3767433316.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  coarse_times[i] = coarse_time_bins[np.abs(coarse_time_bins - these_times[i]) == np.min(np.abs(coarse_time_bins - these_times[i]))]


## burst durations

In [154]:
"""extract individual bursts and its corresponding separation, time, and ap location"""
aps = []
durations = []
event_times = []
for trace in traces:
    t_arr, inten_arr, nucleus = trace
    sub_df = df[df.nucleus_id == nucleus]
    state = binarize_trace(inten_arr, t_arr, thresh=1.0, window_size=3)
    on_times, off_times = get_on_and_off_times(state, t_arr)
    these_burst_durations = get_burst_durations(on_times, off_times)
    these_reference_times = on_times[:len(these_burst_durations)]
    these_aps = np.zeros(len(these_reference_times))
    these_aps_relative_to_somites = np.zeros(len(these_reference_times))
    for i, t in enumerate(these_reference_times):
        these_aps[i] = sub_df[np.abs(sub_df.t - t) == np.min(np.abs(sub_df.t - t))].ap.iloc[0]
  
    aps.extend(these_aps)
    durations.extend(these_burst_durations)
    event_times.extend(these_reference_times)

In [155]:
"""plot durations over time at fixed ap bins"""
these_durations = [x for i, x in enumerate(durations) if aps[i] > lower and aps[i] <= upper]
these_times = [x for i, x in enumerate(event_times) if aps[i] > lower and aps[i] <= upper]

coarse_time_bins = tissue_peaks#np.linspace(16, 119, 4)#10)
coarse_times = np.zeros_like(these_times)
for i in range(len(these_times)):
    coarse_times[i] = coarse_time_bins[np.abs(coarse_time_bins - these_times[i]) == np.min(np.abs(coarse_time_bins - these_times[i]))]
    
mean_durations = np.zeros_like(coarse_time_bins)
std_durations = np.zeros_like(coarse_time_bins)
for i in range(len(coarse_time_bins)):
    tmp_durations = np.array(these_durations)[coarse_times == coarse_time_bins[i]]
    mean_durations[i] = np.nanmean(tmp_durations)
    std_durations[i] = np.nanstd(tmp_durations) #/ np.sqrt(len(tmp_durations))
    
ax = axs[2]
ax.clear()
ax.plot(these_times, these_durations, 'o', markerfacecolor=(0, 0, 0, 0.5), markeredgecolor='none', linewidth=0)
ax.errorbar(coarse_time_bins, mean_durations, std_durations, marker='o',
             markersize=12, color=colors['green'], capsize=4, capthick=2, linewidth=2, markerfacecolor=(colors['green'],) + (0.75,))
#ax.set_xlabel('time (min)', fontsize=fontsize)
ax.set_ylabel('burst duration\n(min)', fontsize=fontsize)
ax.set_xlim([x_min, x_max])
ax.set_xticks([0, 30, 60, 90], labels=[])
ax = style_axes(ax, fontsize=fontsize)


/tmp/ipykernel_2357556/1140779821.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  coarse_times[i] = coarse_time_bins[np.abs(coarse_time_bins - these_times[i]) == np.min(np.abs(coarse_time_bins - these_times[i]))]


## burst amplitudes

In [156]:
aps = []
amplitudes = []
event_times =  []
for trace in traces:
    t_arr, inten_arr, nucleus = trace
    sub_df = df[df.nucleus_id == nucleus]
    state = binarize_trace(inten_arr, t_arr, thresh=1.0, window_size=3)
    on_times, off_times = get_on_and_off_times(state, t_arr)
    
    burst_labels = state[1:] * (1 + np.cumsum(np.diff(state)))
    unique_burst_labels = np.unique(burst_labels[burst_labels > 0])
    n_bursts = len(unique_burst_labels)
    these_mean_intens = np.zeros(n_bursts)
    these_aps = np.zeros(n_bursts)
    these_aps_relative_to_somites = np.zeros(n_bursts)

    for i in range(n_bursts):
        t = t_arr[np.where(burst_labels == unique_burst_labels[i])[0][0]]
        masked_intens = inten_arr[1:] * (burst_labels == unique_burst_labels[i])
        these_mean_intens[i] = np.mean(masked_intens[masked_intens > 0])
        
        these_aps[i] = sub_df[np.abs(sub_df.t - t) == np.min(np.abs(sub_df.t - t))].ap.iloc[0]
   
        event_times.extend([t])

    aps.extend(these_aps)
    amplitudes.extend(these_mean_intens)

In [157]:
these_amplitudes = [x for i, x in enumerate(amplitudes) if aps[i] > lower and aps[i] <= upper]
# rescale for visual ease
these_amplitudes = [x / 5000 for x in these_amplitudes]
these_times = [x for i, x in enumerate(event_times) if aps[i] > lower and aps[i] <= upper]

coarse_time_bins = tissue_peaks#np.linspace(16, 119, 4)#10)
coarse_times = np.zeros_like(these_times)
for i in range(len(these_times)):
    coarse_times[i] = coarse_time_bins[np.abs(coarse_time_bins - these_times[i]) == np.min(np.abs(coarse_time_bins - these_times[i]))]
    
mean_amplitudes = np.zeros_like(coarse_time_bins)
std_amplitudes = np.zeros_like(coarse_time_bins)
for i in range(len(coarse_time_bins)):
    tmp_amplitudes = np.array(these_amplitudes)[coarse_times == coarse_time_bins[i]]
    mean_amplitudes[i] = np.nanmean(tmp_amplitudes)
    std_amplitudes[i] = np.nanstd(tmp_amplitudes) #/ np.sqrt(len(tmp_amplitudes))
    
ax = axs[3]
ax.clear()
ax.plot(these_times, these_amplitudes, 'o', markerfacecolor=(0, 0, 0, 0.5), markeredgecolor='none', linewidth=0)
ax.errorbar(coarse_time_bins, mean_amplitudes, std_amplitudes, marker='o',
             markersize=12, color=colors['green'], capsize=4, capthick=2, linewidth=2, markerfacecolor=(colors['green'],) + (0.75,))
ax.set_xlabel('time (min)', fontsize=fontsize)
ax.set_ylabel('burst amplitude\n(x5000 a.u.)', fontsize=fontsize)
ax.set_xlim([x_min, x_max])
ax.set_xticks([0, 30, 60, 90])
ax = style_axes(ax, fontsize=fontsize)


/tmp/ipykernel_2357556/1215132090.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  coarse_times[i] = coarse_time_bins[np.abs(coarse_time_bins - these_times[i]) == np.min(np.abs(coarse_time_bins - these_times[i]))]


In [158]:
fig.tight_layout()

In [160]:
plt.savefig(r'/home/brandon/Documents/Code/zebrafish-ms2-paper/figures/burst_params_over_time.pdf')